In [ ]:
# Diferenciação do PyTorch

import torch
x = torch.ones(1, 1, requires_grad=True)
print(f"x = {x}")
print(f"x.data = {x.data}")
print(f"x.grad = {x.grad}")
print(f"x.grad_fn = {x.grad_fn}")

In [ ]:
y = x + 2
print(f"y = {y}")
print(f"y.grad_fn = {y.grad_fn}")

In [ ]:
z = y * y * 3
print(z)

In [ ]:
print(x.grad)
z.backward()
print(x.grad)

In [ ]:
  !pip install torchmetrics
# https://torchmetrics.readthedocs.io/en/latest/

import torch
import torch.nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X, y = torch.Tensor(X), torch.Tensor(y)

y = torch.where(y == 0, 0, 1).type(torch.float32)

X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.33, random_state=42)


def standard_scaler(X):
  std, mean = torch.std_mean(X)
  return lambda S: (S - mean) / std

scaler = standard_scaler(X_train)
X_train, X_test = scaler(X_train), scaler(X_test)


1. Construir o modelo;
2. Definir o otimizador;
3. Treinar com *loop* das épocas;

In [ ]:
# construção do modelo
import torch.nn as nn

# mlp com duas camadas, a oculta com 10 neurônios
# a de saída com 1 neurônio

mlp = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Flatten(),
    nn.ReLU(),
)

print(mlp)

Esta é uma lista de alguns otimizadores comuns:
- ```torch.optim.Adam()```: O algoritmo de Adam (estimativa do momento adaptativo);
- ```torch.optim.NAdam()```: O algoritmo de Adam com momento de Nesterov;
- ```torch.optim.SGD()```: Descida de gradiente estocástico;
- ```torch.optim.RMSprop()```: O algoritmo RMSprop.

In [ ]:
# Definir o otimizador
otimizador = torch.optim.Adam(mlp.parameters(), lr=0.001)
otimizador

Abaixo estão algumas funções de perda comuns no PyTorch:
 - ```nn.MSELoss()```: erro quadrático médio, útil em problemas de regressão;
 - ```nn.CrossEntropyLoss()```: perda de entropia cruzada, útil em problemas de classificação multi-classe;
 - ```nn.BCELoss()```: perda de entropia cruzada binária, útil em problemas de classificação binária.

In [ ]:
# loop de treinamento
epocas = 50
funcao_custo = nn.BCELoss()

# gradiente descendente em lote cheio
for n in range(epocas):
  y_pred = mlp(X_train)
  custo = funcao_custo(y_pred.squeeze(), y_train)
  otimizador.zero_grad()
  custo.backward()
  otimizador.step()

# rede treinada!



In [ ]:
y_pred = mlp(X_test).squeeze()
custo_teste = funcao_custo(y_pred, y_test)

print(f"Último custo de treino: {custo}")

print(f"Custo de teste {custo_teste}")




In [ ]:
from torchmetrics import Accuracy

accuracy = Accuracy(task="binary")
accuracy(y_pred, y_test)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

f"Nossa mlp possui {count_parameters(mlp)} parâmetros treináveis."


In [ ]:
# Preceptron
class Perceptron(torch.nn.Module):
    def __init__(self, input_size, epochs=400, lr=0.001):
        super().__init__()
        self.epochs = epochs
        self.lr = lr
        self.fc = nn.Linear(input_size, 1)
        self.relu = torch.nn.Sigmoid() # instead of Heaviside step fn

    def count_parameters(self):
      return sum(p.numel() for p in self.parameters() if p.requires_grad)


    def forward(self, x):
        output = self.fc(x)
        output = self.relu(output) # instead of Heaviside step fn
        return output

    def fit(self, X, y):
      funcao_custo = nn.BCELoss()

      otimizador = torch.optim.Adam(self.parameters(), lr=self.lr)


      # gradiente descendente em lote cheio
      for n in range(self.epochs):
        otimizador.zero_grad()

        y_pred = self.predict(X)

        custo = funcao_custo(y_pred, y)

        custo.backward()
        otimizador.step()

    def predict(self, X):
      return self.forward(X).squeeze()

    def score(self, X, y):
      from torchmetrics import Accuracy

      accuracy = Accuracy(task="binary")
      return accuracy(self.predict(X), y)


p = Perceptron(4)
p.fit(X_train, y_train)
print(f"Acuracia: {p.score(X_test, y_test)}")
print(f"N params {p.count_parameters()}")

# https://colab.research.google.com/github/bentrevett/pytorch-image-classification/blob/master/1_mlp.ipynb#scrollTo=lAqzcW9XREvu